# PRIO Linux: GPIO Demo
This demo illastrates how to use device tree overlays to interact with partially reconfigurable designs. 


### Step One: 
Download the static bitstream

In [1]:
from prio_linux import PrIoOverlay

FULL_BITSTREAM_PATH = "/usr/local/lib/python3.6/dist-packages/prio_linux/"
PARTIAL_BITSTREAM_PATH = "/usr/local/lib/python3.6/dist-packages/prio_linux/partial/"
DTBO_PATH = "/usr/local/lib/python3.6/dist-packages/prio_linux/dtbo/"

overlay = PrIoOverlay(FULL_BITSTREAM_PATH + "prio_linux.bit")

### Step Two:
Download the partial bitstreams and insert the dtbo files

In [2]:
overlay.pr_download("pr_0", PARTIAL_BITSTREAM_PATH + "pr_0_gpio.bit", DTBO_PATH + "pr_0_gpio.dtbo")

### Step Three:

Define a convenient wrapper class that wraps around Xilinx's GPIO wrapper class (reminds me of a certain Christopher Nolan movie, except instead of dreams, it's GPIO wrappers) and groups pins into a single GPIO object.

In [3]:
from pynq import GPIO  

# returns the bit of the value at the position            
def getBit(value, position):
    return (value & (0x01 << position)) >> position

# puts bit in value at postion
def putBit(value, bit, position):
    return (value | (int(bit) << position))

    
class BYU_GPIO:
    """Class to group GPIO pins together into a user-friendly interface.
    Attributes
    ----------
    starting_index : int
        The index of the first GPIO pin
    num_of_pins: int
        number of pins, starting from the gpio pin at the starting index, that you 
        would like to group together.
    direction : int
        Input/output direction of the GPIOs. the LSB corresponding to the first index.
        0 signifies output, 1 siginifies input.
    
    """
    
    ALL_INPUTS = 0xFFFF
    ALL_OUTPUTS = 0x0000
    
    def __init__(self, name, direction):
        name = "/amba/" + name
        self.pins = []
        for x in range(GPIO.get_gpio_npins(name)):
            if (getBit(direction, x)):
                direction_str = 'in'
            else:
                direction_str = 'out'
            self.pins.append(GPIO(GPIO.get_gpio_pin(x, name), direction_str))
            
    def setDirection(self, direction):
        for x in range(0, len(self.pins)):
            if (getBit(direction, x)):
                direction_str = 'in'
            else:
                direction_str = 'out'   
            with open(self.pins[x]._impl.path + 'direction', 'w') as f:
                f.write(direction_str)
            self.pins[x]._impl.direction = direction_str
            
            
    def getDirection(self):
        direction = 0
        for x in range(0, len(self.pins)):
            if (self.pins[x].direction == "in"):
                direction = putBit(direction, 1, x)
        return direction  
    
    def write(self, value):
        for x in range(0, len(self.pins)):
            if(self.pins[x].direction == "out"):
                self.pins[x].write(getBit(value, x))
            else:
                pass
#                 print("WARNING (GPIO_PIN" + str(self.pins[x].index) + "): Cannot write GPIO input.")
    
    def read(self):
        value = 0
        for x in range(0, len(self.pins)):
            if(self.pins[x].direction == "in"):
                value = putBit(value, self.pins[x].read(), x)
            else:
                pass
#                 print("WARNING (GPIO_PIN" + str(self.pins[x].index) + "): Cannot read GPIO output.")
        return value
    
    def release(self):
        for pin in(self.pins):
            pin.release()

### Step Four:

Initialize the upper pins of **`PMOD0`** for the gpio hardware inside pr_region_0.

In [4]:
gpio0 = BYU_GPIO("pr_0_gpio", BYU_GPIO.ALL_OUTPUTS)

### Step Five:

Run the demos!

## Demo: Constant Output
Outputs a constant pattern to the **`pr_0`** pins.

** Hardware Setup:** For this demo, we suggest using at least one LED to check the output of every pin. These LEDs should be hooked up to the top row of **`PMOD 0`**. Every other pin should be high.

<img src="images/gpio_hw_setup1.JPG" align="center"/>


In [5]:
# Desired pattern of gpio output.
LED_PATTERN = 0b1010

#sets the value of all the pins in region 0 so everyother pin is high.
gpio0.write(LED_PATTERN)

## Demo: Binary Counter
This demo will output the count, in binary, of a counter to the pins of **`pr_0`**.

** Hardware Setup:** This Demo should use the same LED setup as the **`Constant Output`** Demo

In [6]:
import time

OUTPUTS_OFF = 0b0

# Sets the direction of all pins in region 0 to  output.
gpio0.setDirection(BYU_GPIO.ALL_OUTPUTS)

# Counts from 0 to 15 in binary
for output in range(0, 16):
    
    gpio0.write(output)
    
    #pauses for a half second
    time.sleep(0.5)
    
#sets the value of all the pins in region 0 so every pin is low
gpio0.write(OUTPUTS_OFF)

## Demo: Blink Test
The following demonstrates the Input and Output functionalities of GPIO interface by toggling the value on top-right pin on **`PMOD 1`**, and reading that value from the bottom-right pin on **`PMOD 1`**. This value will then be echoed to the top-right pin on **`PMOD 0`**.

** Hardware Setup:** Keep the LEDs on the top row of **`PMOD 0`**. Connect a wire from the top-right pin on **`PMOD 1`** to the bottom-right pin on **`PMOD 1`**.



<img src="images/gpio_hw_setup2.JPG" align="center"/>


The LED connected to the top-right pin on PMOD 0 will indicate the current value being read from bottom-right pin on PMOD 1. This LED should toggle on and off.

### Setup additional reconfigurable regions
Since this demo requires two additional PR regions, we must setup them up with the following code.

In [7]:
overlay.pr_download("pr_1", PARTIAL_BITSTREAM_PATH + "pr_1_gpio.bit", DTBO_PATH + "pr_1_gpio.dtbo")
gpio1 = BYU_GPIO("pr_1_gpio", BYU_GPIO.ALL_OUTPUTS)

overlay.pr_download("pr_3", PARTIAL_BITSTREAM_PATH + "pr_3_gpio.bit", DTBO_PATH + "pr_3_gpio.dtbo")
gpio3 = BYU_GPIO("pr_3_gpio", BYU_GPIO.ALL_INPUTS)

In [8]:
import time

# Since only the right most pin is needed as an output
# for the LEDs, that one is set to an output(0), while
# the rest are set to inputs (1).
gpio0.setDirection(0xe)

led_value = 0

for i in range(20):
    time.sleep(0.5)
    gpio1.write(led_value)
    led_value = led_value ^ 0b10000001
    
    val = gpio3.read() & 0x01
    
    gpio0.write(val)
    if val:
        print('on')
    else:
        print('off')

off
on
off
on
off
on
off
on
off
on
off
on
off
on
off
on
off
on
off
on


In [9]:
gpio0.release()
gpio1.release()
gpio3.release()